# Exercícios: Pré-processamento
Fonte: CARVALHO, Fabrício Galende Marques de. **Notas de aula da disciplina processamento de linguagem natural.** São José dos Campos, 2023.

#### Atividade realizada em grupo: 
- Gabriel Camargo Leite
- Giovana Thaís de O. Silva
- Isabelle Dias R. Silva
- João Marcos O. Santos
- Maria Gabriela G. S. Reis
- Thiago Henrique Ferreira

## Terminologia e conceitos
---

1.  (TC.2.2) Qual um possível efeito da não remoção de um iframe ou de scripts e CSS’s de um documento capturado através de uma raspagem de dados? 

**Resposta:** Conforme consultado no material de aula, caso a remoção de tags, executada depois da raspagem de dados em rede (web scraping), não seja feita, ou seja, se as tags e dados não informativos (scripts, css, iframes…) não forem removidos do texto, é possível que ruídos sejam criados na base de dados a ser utilizada pelas próximas etapas da pipeline, o que poderia influenciar em análises errôneas.

2. (TC.2.4) Cite exemplos de tokens de frases e tokens de palavras que podem significar:

| Situação                                                                          | Token de frase             | Token de palavra |
|:----------------------------------------------------------------------------------|:---------------------------|:-----------------|
| Opinião negativa referente a um produto de uma loja de vestuário                  | Tecido ruim                | Largo            |
| Opinião positiva referente a um produto de uma loja de vestuário                  | Costura muito boa          | Confortável      |
| Opinião neutra referente a um produto de uma loja de vestuário                    | Veste mais ou menos        | OK               |
| Opinião negativa relacionada a um carro vendido por uma concessionária automotiva | Câmbio impreciso           | Desalinhado      |
| Opinião positiva relacionada a um carro vendido por uma concessionária automotiva | Manutenção simples         | Robustez         |
| Opinião neutra relacionada a um carro vendido por uma concessionária automotiva   | Carro bom, mas gasta muito | Decente          |


## Prática de programação
---

1. (PP.2.1) Tomando como base o código-fonte fornecido pelo professor e efetuando uma raspagem de dados que opere sobre alguma página contendo revisão de produtos, efetue uma comparação de desempenho utilizando 3 modelos de tokenizadores de sentença. Qual sua conclusão?

> **OBS:** Kaggle não deixa pegar a review do Mercado Livre (proteção contra requisições)

In [ ]:
# =============================================================================
# Instalação do pacote em Português do Spacy
# =============================================================================
!python -m spacy download pt

In [ ]:
from colorama import Fore, Back, Style 
import requests
from bs4 import BeautifulSoup
import time
from transformers import AutoTokenizer
import nltk
#nltk.download('punkt')
import spacy
nlp = spacy.load("pt_core_news_sm")

def format_time(seconds):
    if seconds < 1:
        return f"{seconds * 1000:.2f} ms"
    else:
        return f"{seconds:.2f} s"
    
def spacy_tokenizer(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

# a function to get rid of html tags
def get_rid_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    # iframe and script nodes removal from doc tree
    [s.extract() for s in soup(['iframe','script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]','\n',stripped_text)
    return stripped_text

def cus_data(soup):
    # find the Html tag with find() and convert into string
    data_str = ""
    cus_list = []
  
    for item in soup.find_all("p", class_="ui-review-capability-comments__comment__content"):
        data_str = data_str + item.get_text()
        cus_list.append(data_str)
        data_str = ""
    return cus_list


data = requests.get("https://www.mercadolivre.com.br/controle-joystick-sem-fio-sony-playstation-dualsense-edge-branco/p/MLB21286427?pdp_filters=category:MLB455266#reviews")

content = data.content
soup = BeautifulSoup(content, "html.parser")

cus_res = cus_data(soup)
print("\n=============================\nAvaliações do controle de PS5\n=============================\n")

for y in range(0, len(cus_res)):
    print(f'{y+1}: {cus_res[y]}')
    review_text = cus_res[y]
    
    # NLTK_TOKENIZER
    start_time = time.time()
    nltk_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    default_nltk_tokens = nltk_tokenizer.tokenize(review_text)
    nltk_elapsed_time = time.time() - start_time
    print("=================================================================")

    print(f'Tokens {Fore.BLUE}NLTK (padrão){Fore.RESET}: {default_nltk_tokens}')
    print(f'{Fore.GREEN} Tempo de processamento {Fore.BLUE}NLTK (padrão){Fore.GREEN}: {format_time(nltk_elapsed_time)} {Fore.RESET}')
    print("- - - - - - - - - - -")

    # Tokenizador simples (espaço como delimitador)
    start_time = time.time()
    simple_tokens = review_text.split()  # Usando split() diretamente
    simple_elapsed_time = time.time() - start_time

    print(f'Tokens com {Fore.BLUE}delimitador de espaço{Fore.RESET}: {simple_tokens}')
    print(f'{Fore.GREEN} Tempo de processamento com {Fore.BLUE} delimitador de espaço {Fore.GREEN}: {format_time(simple_elapsed_time)} {Fore.RESET}')
    print("- - - - - - - - - - -")
    
    # Medindo o desempenho do tokenizador spaCy
    start_time = time.time()
    spacy_tokens = spacy_tokenizer(review_text)
    spacy_elapsed_time = time.time() - start_time
    
    print(f'Tokens {Fore.BLUE}spaCy{Fore.RESET}: {spacy_tokens}')
    print(f'{Fore.GREEN} Tempo de processamento {Fore.BLUE}spaCy{Fore.GREEN}: {format_time(spacy_elapsed_time)} {Fore.RESET}')
    print("- - - - - - - - - - -")
    
    # Medindo o desempenho do tokenizador HuggingFace Transform
    start_time = time.time()
    transformers_tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
    hugging_tokens = transformers_tokenizer.tokenize(review_text)
    hugging_elapsed_time = time.time() - start_time

    print(f'Tokens {Fore.BLUE}HuggingFace{Fore.RESET}: {hugging_tokens}')
    print(f'{Fore.GREEN} Tempo de processamento {Fore.BLUE}HuggingFace{Fore.GREEN}: {format_time(hugging_elapsed_time)} {Fore.RESET}') 
    
    print("=================================================================\n\n\n\n")
    
print("\n************ END ************")

2. (PP.2.4) Exemplifique o funcionamento de um token de palavras utilizando expressão regular. Explique, com um programa exemplo, como configurar o padrão para obter um comportamento diferente do tokenizador. 

In [ ]:
import nltk
from nltk.tokenize import regexp_tokenize

frase = 'Tornar o futuro seguro e Próspero, conectando ciência e Tecnologia!'

# Frase retirando espaços
tokenizacao1 = regexp_tokenize(frase, r'\s+', gaps=True)

# Frase selecionando caracter maiúsculo 
tokenizacao2 = regexp_tokenize(frase, r'[A-Z]\w+')

print(tokenizacao1)
print('---------------------------------------------------------')
print(tokenizacao2)

3. (PP.2.6) Exemplifique o funcionamento de um corretor ortográfico, aplicável à língua portuguesa, que efetue correção de palavras baseado em um corpus de texto considerado como referência e que utilize métricas de distância e estatísticas de ocorrência de palavras no corpus considerado. Alterar o corpus pode afetar o comportamento do corretor? Se sim, dê um exemplo prático utilizando dados diferentes para o corpus.

4. (PP.2.7) Aplique técnicas de tokenização e correção de erros de ortografia a dados de revisão de produtos que tenham sido raspados de uma página de revisão da Internet. Ilustre o comportamento e o desempenho do seu trecho de pipeline de PLN identificando os principais gargalos e sugira uma melhoria possível. Esclareça o porquê da ordem dos elementos em sua pipeline.

In [ ]:
# Importando bibliotecas
import pandas as pd
import nltk
from nltk import tokenize    
from nltk.corpus import words
from nltk.metrics.distance import edit_distance

# Definindo frase completa
frase = "Good resolutionn quality hoewever the shape of the packging left somethinggg to be desirred"

# Tokenizando a frase
tokens = tokenize.word_tokenize(frase)
print(tokens)
print('----------------------------------------------------------------')

# Importando palavras corretas
palavras_corretas = words.words()
print(palavras_corretas[:20])
print('----------------------------------------------------------------')

data = []

for token in tokens:
    menorDistancia = {"distancia": 100, "palavra": ""}
    
    for palavra in palavras_corretas:
        distancia = edit_distance(token, palavra)
        if distancia < menorDistancia["distancia"]:
            menorDistancia["distancia"] = distancia
            menorDistancia["palavra"] = palavra
   
    palavra_corrigida = [menorDistancia["palavra"], menorDistancia["distancia"]]
    data.append(palavra_corrigida)

print(data)
print('----------------------------------------------------------------')

# for token in tokens:
#     temp = [(edit_distance(token, palavra), palavra) for palavra in palavras_corretas if palavra[0]==token[0]]
#     print(sorted(temp, key = lambda val:val[0])[0][1])

df = pd.DataFrame(data,columns=['Palavra corrigida','Distância'])
df.head(50)